### Extract (Extração)

In [2]:
import pandas as pd

# Caminho dos diretórios
dir_importados = 'arquivos_importados'
dir_relatorios = 'relatorios'

df_candidatos_selecionados = pd.read_excel(f"{dir_importados}/Ciclo1JovemPotiguar_VersaoCompleta.xlsx")
cursos_ids = df_candidatos_selecionados['jp_id_curso'].unique().tolist()
print(cursos_ids)

[64, 65, 83, 60, 75, 54, 52, 84, 67, 71, 85, 53, 62, 55, 81, 58, 69, 82, 72, 76, 61, 74]


### Transform (Transformação ou Enriquecimento)

In [ ]:
import numpy as np

def buscar_curso(curso_id):
    df_curso = df_candidatos_selecionados[df_candidatos_selecionados['jp_id_curso'] == curso_id]

    # Condições para o candidato ser considerado ELEGÍVEL:
    # (1) Selecionar a idade entre 16 e 29 anos, exceto nos cursos que consideram entre 18 e 29 anos
    condicao1 = (df_curso['idade_inicio_curso'] >= 15.5) & (df_curso['idade_inicio_curso'] < 30)
    condicao2 = (df_curso['jp_nome_curso'] == 'Auxiliar de Manutenção Predial') | (df_curso['jp_nome_curso'] == 'Eletricista de Sistemas de Energias Renováveis')
    condicao3 = (df_curso['idade_inicio_curso'] >= 17.5) & (df_curso['idade_inicio_curso'] < 30)
    # (2) Pertencer ao CadÚnico
    condicao4 = df_curso['cadunico_nis'] > 0
    # (3) Ter concluído o Ensino Fundamental. Todos que se inscreveram autodeclaram ter concluído o Ensino Fundamental
    
    if not df_curso.empty:
        dict_curso = {
            'ID CURSO': curso_id,
            'NOME CURSO': df_curso['jp_nome_curso'].unique().tolist()[0],
            'CAMPUS': df_curso['jp_campus'].unique().tolist()[0],
            'VAGAS': int(np.sum(df_curso['Situação'] == 'APROVADO')),
            'DATA INÍCIO': pd.to_datetime(df_curso['jp_inicio_curso'].unique().tolist()[0]).strftime('%d/%m/%Y'),
            'TURNO' : df_curso['jp_turno_curso'].unique().tolist()[0],
            'INSCRITOS': len(df_curso['jp_id_curso']),
            'ELEGÍVEIS': int(np.sum(((condicao1 & ~condicao2) | (condicao2 & condicao3)) & condicao4)),
            'IDADE VÁLIDA': int(np.sum((condicao1 & ~condicao2) | (condicao2 & condicao3))),
            'NIS NO CADUNICO': int(np.sum(condicao4)),
            'EGRESSOS NA INSCRIÇÃO': int(np.sum(df_curso['jp_egresso'] == 'Sim')),
            'EGRESSOS NA FUNDASE': int(np.sum(df_curso['fundase_cpf'] == 'Sim')),
            'PRETOS E PARDOS': int(np.sum((df_curso['cadunico_etnia'] == 'Preta') | (df_curso['cadunico_etnia'] == 'Parda'))),
            'MULHERES': int(np.sum(df_curso['cadunico_sexo'] == 'Feminino')),
            'PCDs': int(np.sum(df_curso['cadunico_deficiencia'] == 'Sim')),
            'TRANSEXUAIS': int(np.sum(df_curso['jp_trans'] == 'Sim')),
            'DOC COOPERATIVA': int(np.sum(df_curso['Doc válido'] == 'Sim'))
        }
        return dict_curso
    else:
        return f"Curso com ID {curso_id} não encontrado."
    

cursos = [curso for id in cursos_ids if (curso := buscar_curso(id)) is not None] # ':=' permite concatenar os cursos em uma lista
print(cursos)

### Load (Carregamento)

In [4]:
pd.DataFrame(cursos).to_excel(f"{dir_relatorios}/cursos_lista.xlsx", index=False)
print('Exportação concluída!')

Exportação concluída!
